In [13]:
import os.path
!pip install datasets transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


  Obtaining dependency information for kmedoids from https://files.pythonhosted.org/packages/6e/b4/457e552aa362cc746fbf0c461ab1a9bd7794db3321f82586601769f27112/kmedoids-0.5.3.1-cp312-cp312-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 kB 9.3 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Setup

In [1]:
%load_ext autoreload
%autoreload 2

from transformers import ClapModel, ClapFeatureExtractor, AutoTokenizer

device='mps'
model = ClapModel.from_pretrained("laion/clap-htsat-unfused", use_safetensors=True).to(device)
feature_extractor: ClapFeatureExtractor = ClapFeatureExtractor.from_pretrained("laion/clap-htsat-unfused")
tokenizer = AutoTokenizer.from_pretrained("laion/clap-htsat-unfused")


# Run

In [2]:
import torchaudio
import torchaudio.transforms as T

filename = '/Users/damian/Downloads/Bon Iver - Michicant (Deluxe) - Official Video.mp3'
waveform, sampling_rate = torchaudio.load(filename)
chunk_beats = 2
bpm = 161/2
window_width = 0

if sampling_rate != feature_extractor.sampling_rate:
    resampler = T.Resample(sampling_rate,  feature_extractor.sampling_rate, dtype=waveform.dtype)
    waveform, sampling_rate = resampler(waveform), feature_extractor.sampling_rate

print(sampling_rate, waveform.shape)
#inputs = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt")
#audio_features = model.get_audio_features(**inputs)

48000 torch.Size([2, 11764158])


In [3]:
from typing import Generator
import torch
torch.set_grad_enabled(False)
from tqdm.auto import tqdm

def wrap(idx, total):
    if idx < 0:
        return total + idx
    elif idx >= total:
        return idx - total
    else:
        return idx
    
def get_audio_chunks(waveform, chunk_size_seconds: float=0.25, 
                     window_width: float=0, include_window: bool=True
             ) -> Generator[torch.Tensor, None, None]:
    if len(waveform.shape) != 1:
        raise ValueError("waveform should have shape [num_samples]")
    chunk_size = int(8 * ((sampling_rate * chunk_size_seconds) // 8))
    window_width_samples = int(chunk_size * window_width)
    for offset in range(window_width_samples, waveform.shape[0]-window_width_samples, chunk_size):
        if include_window:
            start = wrap(offset-window_width_samples, waveform.shape[0])
            end = wrap(offset+chunk_size+window_width_samples, waveform.shape[0])
        else:
            start = offset
            end = offset + chunk_size
        yield waveform[start:end]
    

def get_audio_features(waveform: torch.Tensor, sampling_rate):
    inputs = feature_extractor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    #print(inputs.keys())
    audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
    return audio_features / torch.norm(audio_features, p=2, dim=1, keepdim=True)

def get_text_features(text: str):
    inputs = tokenizer(text, padding=True, return_tensors='pt')
    text_features =  model.get_text_features(input_ids = inputs.input_ids.to(device))
    return text_features / torch.norm(text_features, p=2, dim=1, keepdim=True)


In [4]:
import pickle
import os

chunk_size_seconds = chunk_beats*60/bpm
left_chunks_window = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds, window_width=window_width))[:-1]
right_chunks_window = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds, window_width=window_width))[:-1]
mono_chunks = [(left_chunks_window[i] + right_chunks_window[i]) / 2 for i in range(len(left_chunks_window))]

left_chunks_no_window = list(get_audio_chunks(waveform[0], chunk_size_seconds=chunk_size_seconds, window_width=window_width, include_window=False))[:-1]
right_chunks_no_window = list(get_audio_chunks(waveform[1], chunk_size_seconds=chunk_size_seconds, window_width=window_width, include_window=False))[:-1]
stereo_chunks_no_window = [torch.stack([left_chunks_no_window[index], right_chunks_no_window[index]])
    for index in range(len(left_chunks_no_window))]
features_pickle_filename = filename + f'.clap-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}.pkl'
if os.path.exists(features_pickle_filename):
    with open(features_pickle_filename, 'rb') as f:
        all_features = pickle.load(f)
else:
    all_features = torch.concat([get_audio_features(chunk, sampling_rate=sampling_rate) 
                             for chunk in tqdm(mono_chunks)])
    with open(features_pickle_filename, 'wb') as f:
        pickle.dump(all_features, f)



  0%|          | 0/164 [00:00<?, ?it/s]

/Users/damian/2.current/clapSlice/venv/lib/python3.12/site-packages/torch/nn/functional.py:4046: UserWarning: The operator 'aten::upsample_bicubic2d.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._C._nn.upsample_bicubic2d(input, output_size, align_corners, scale_factors)


In [5]:
from medoids_tsp import sort_tsp
sort_order = sort_tsp(all_features)

computing distance matrix
computing route


In [6]:
chunks_sorted = [torch.stack([left_chunks_no_window[index], right_chunks_no_window[index]])
    for index in sort_order]
print(chunks_sorted[0].shape)
sorted_audio = torch.cat(chunks_sorted, dim=1)
print(sorted_audio.shape)

torch.Size([2, 71552])
torch.Size([2, 11734528])


In [7]:
sort_order

tensor([ 25, 100, 115, 117, 104, 109, 103, 102, 116, 110, 123,  51, 126,  74,
         75,  58,  41,  28,  21,  22,  42,  36,  29,  35,  77,  78,  76,  71,
         57,  70,  65,  52,  64, 151, 133, 134, 152, 142, 149, 144, 148, 146,
        155, 156, 153, 145, 150, 147, 141, 139, 154, 136, 138, 132, 137, 143,
        140, 135, 129, 128, 130, 158, 157,  97,  79,  86,  82,  88,  85,  83,
         91,  90,  89,  87,  92, 122, 127,  95,  96,  94,  84,  93,  81,  80,
        159, 160, 161, 162, 163,   0,   1,   3,   4,   8,  10,  11,   9,   7,
          6,   5,  12,   2,  14,  15,  16,  46,  72,  50, 131,  43, 105, 118,
         63,  30,  34,  23,  13,  45,  20,  40,  27,  59, 111,  26,  44, 101,
         49,  47,  62, 107,  55,  66,  53,  68, 125,  73, 124, 121, 108,  56,
         69,  39,  98, 113,  61,  32,  38,  24,  99,  17,  18, 106, 112,  60,
        119,  37,  31,  67,  33, 114, 120,  19,  48,  54])

In [204]:
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}.wav', sorted_audio, sample_rate=sampling_rate)

In [10]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            smear_width = torch.sum((logits / logits.sum()) 
                                              * torch.tensor(dynamic_smear_width_embeds[1]).to(device)).item()
            print(smear_width)
            smear_width = max(round(smear_width), 1)
            envelope = get_envelope(smear_width)
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*envelope.sum()))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6), ('intense, emotional, screaming, wailing', 1)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(s) for s, _ in dynamic_smear_width_defs]),
    [w for _, w in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              #dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/164 [00:00<?, ?it/s]

torch.Size([2, 11734528])


In [15]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int], list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            logits_norm = logits / logits.sum()
            smooth_dynamic_smear = True
            if smooth_dynamic_smear:
                smear_width = torch.sum(
                    logits_norm 
                    * torch.tensor(dynamic_smear_width_embeds[1]).to(device)
                ).item()
                spread = torch.sum(
                    logits_norm 
                    * torch.tensor(dynamic_smear_width_embeds[2]).to(device)
                ).item()
                print(smear_width, spread)
                smear_width = max(round(smear_width), 1)
                spread = max(round(spread), 1)
            else:
                smear_width = dynamic_smear_width_embeds[1][int(logits_norm.argmax().item())]
                spread = dynamic_smear_width_embeds[2][int(logits_norm.argmax().item())]
                print(smear_width, spread)
            envelope = get_envelope(smear_width)
        #volume_scaling = 1/envelope.sum()
        
        noclip_ramp = 200
        chunk_size_samples = original_order_chunks_no_window[0].shape[1]
        spread_envelope_in = torch.cat([torch.linspace(0, 1, noclip_ramp),
                                            torch.ones(chunk_size_samples-noclip_ramp)])
        spread_envelope_out = torch.cat([torch.ones(chunk_size_samples-noclip_ramp), 
                                                 torch.linspace(1, 0, noclip_ramp)])
        spread_envelope_flat = torch.ones(chunk_size_samples)
        spread_envelope = [spread_envelope_in] + [spread_envelope_flat] * (2*spread-1) + [spread_envelope_out]
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*envelope.sum()))
                        * spread_envelope[spread_slot+spread]
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
spread = 4
smear_width = 1
smear_mode = 'in-out'

dynamic_smear_width_defs = [('a tenor voice sings softly, mysterious, sparse', 4, 1), 
                            ('gentle acoustic jazz instruments make a lovely soundscape', 1, 4), 
                            #('intense, emotional, screaming, wailing', 1, 6)
                            ]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(t) for t, _, _ in dynamic_smear_width_defs]),
    [w for _, w, _ in dynamic_smear_width_defs],
    [s for _, _, s in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn3a-env.wav', smeared, sample_rate=sampling_rate)


tensor([[    0, 40110,     2]])
tensor([0., 1., 0.])


  0%|          | 0/164 [00:00<?, ?it/s]

2.343362808227539 2.656637191772461
2.642634391784668 2.357365369796753
1.8989572525024414 3.1010427474975586
1.7994635105133057 3.2005367279052734
1.7540531158447266 3.2459468841552734
1.5838580131530762 3.416142463684082
1.723629117012024 3.2763710021972656
1.6055140495300293 3.3944857120513916
1.7240736484527588 3.275926113128662
1.7892026901245117 3.21079683303833
1.6229168176651 3.3770830631256104
1.6846914291381836 3.3153088092803955
2.03239107131958 2.96760892868042
2.048370361328125 2.951629638671875
2.1246886253356934 2.8753111362457275
1.8424794673919678 3.1575207710266113
1.9157613515853882 3.0842387676239014
2.0313029289245605 2.9686973094940186
2.186267614364624 2.813732385635376
2.3249452114105225 2.6750547885894775
2.3317489624023438 2.6682510375976562
2.2716400623321533 2.7283599376678467
2.339428663253784 2.6605710983276367
2.4336161613464355 2.5663836002349854
2.4699482917785645 2.5300517082214355
2.5218658447265625 2.4781341552734375
2.3601691722869873 2.639830827713

In [210]:
torch.tensor([1, 2, 3]).argmax()

tensor(2)

In [ ]:
from typing import Literal
import math

def smear(chunks, smear_width):
    envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
    #for i in range(smear_width+1, 2*smear_width+1):
    #    envelope[i] = 0
    print(envelope)
    for i in range(len(chunks)-smear_width*2-1):
        smear_sources = [
            chunks[i+chunk_index] * envelope[chunk_index]
            #(i+chunk_index, envelope[chunk_index])
            for chunk_index in range(0, min(len(chunks), smear_width*2+1))
        ]
        #print(smear_sources)
        #print(set([ss.shape for ss in smear_sources]))
        #print(torch.sum(torch.stack(smear_sources), dim=0))
        smeared_chunk = torch.sum(torch.stack(smear_sources), dim=0) / len(smear_sources)
        #print(smeared_chunk.shape)
        #break
        yield smeared_chunk
        
def smear_2(original_order_chunks_no_window, sort_order, smear_width, spread=4, 
            smear_mode=Literal['in', 'in-out'],
            dynamic_smear_width_embeds: tuple[torch.Tensor, list[int], list[int]]=None):
    
    def get_envelope(smear_width):
        envelope = 0.5*(1-torch.cos(torch.linspace(0, 2*math.pi, 2*smear_width+1)))
        if smear_mode == 'in':
            for i in range(smear_width+1, 2*smear_width+1):
                envelope[i] = 0
        return envelope
    envelope = get_envelope(smear_width)
    print(envelope)
    num_chunks = len(original_order_chunks_no_window)
    smeared_chunks = [torch.zeros_like(original_order_chunks_no_window[0]) 
                     for _ in range(num_chunks)]
    for position_idx in tqdm(range(num_chunks), total=num_chunks):
        source_chunk_idx = sort_order[position_idx]
        
        if dynamic_smear_width_embeds is not None:
            source_chunk_embed = all_features[source_chunk_idx]
            logits = source_chunk_embed @ dynamic_smear_width_embeds[0].T
            #print(logits)
            #print(logits.softmax(dim=0))
            logits_norm = logits / logits.sum()
            smear_width = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[1]).to(device)
            ).item()
            spread = torch.sum(
                logits_norm 
                * torch.tensor(dynamic_smear_width_embeds[2]).to(device)
            ).item()
            print(smear_width, spread)
            smear_width = max(round(smear_width), 1)
            spread = max(round(spread), 1)
            envelope = get_envelope(smear_width)
                    
        for smear_slot in range(-smear_width, smear_width+1):
            for spread_slot in range(-spread, spread+1):
                smeared_chunks[
                    wrap(position_idx+smear_slot+spread_slot, num_chunks)
                ] += (
                        original_order_chunks_no_window[
                            wrap(source_chunk_idx+spread_slot, num_chunks)]
                        * envelope[smear_slot+smear_width] 
                        * (1/((spread+1)*smear_width))
                )

    for smeared_chunk in smeared_chunks:
        yield smeared_chunk
        
    #for i 
smear_width = 1
spread = 4
smear_mode = 'in-out'

dynamic_smear_width_defs = [('dreamy, mysterious, sparse', 6, 4), 
                            ('driving, rhythmic, instrumental, simple beat', 2, 2), 
                            ('intense, emotional, screaming, wailing', 1, 6)]
print(tokenizer('stuff', padding=True, return_tensors='pt').input_ids)

dynamic_smear_width_embeds = (
    torch.concat([get_text_features(t) for t, _, _ in dynamic_smear_width_defs]),
    [w for _, w, _ in dynamic_smear_width_defs],
    [s for _, _, s in dynamic_smear_width_defs]
)

smeared_chunks = list(smear_2(stereo_chunks_no_window, sort_order=sort_order, smear_width=smear_width, spread=spread, smear_mode=smear_mode, 
                              dynamic_smear_width_embeds=dynamic_smear_width_embeds
                              ))
#print(Counter([len(c) for c in smeared_chunks]))
smeared = torch.cat(smeared_chunks, dim=1)
print(smeared.shape)
torchaudio.save(filename + f'-sorted-norm-bpm{bpm}-cb{chunk_beats}-ww{window_width}-smeared-sm_{smear_mode}-sw{smear_width}-spread_actual{spread}-dyn3b.wav', smeared, sample_rate=sampling_rate)
